In [21]:
import numpy as np
import os
import time
import torch
from torch import nn
from torch.utils.data import TensorDataset, DataLoader
%matplotlib widget
import matplotlib.pyplot as plt
import scipy.io
from utilities import *

In [22]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cpu device


Import data from Matlab

In [23]:
mat = scipy.io.loadmat('simple_mlp_data_10k.mat')
A_eq = mat.get('A_eq')
b_eq = mat.get('b_eq')
A_ieq = mat.get('A_ieq')
b_ieq = mat.get('b_ieq')
N = mat.get('N')
P = mat.get('P')
R0_vec = mat.get('R0_vec')
PN = torch.tensor(P@N).float()
Pr0 = torch.tensor(P@R0_vec).float()

# some parameter for objective function
#A_ee = torch.tensor(mat.get('A_ee')).float()
#b_ee = torch.tensor(mat.get('b_ee')).float()

# separate train and test sets
X_train = mat.get('ee_poses')[:,0:5000].T
R_vec_allcol = mat.get('R_configs')
R_vec_2col = P@R_vec_allcol
y_train = R_vec_2col[:,0:5000].T
X_test = mat.get('ee_poses')[:,5000:].T
y_test = R_vec_2col[:,5000:].T
m = X_train.shape[1]
m_vr = np.linalg.matrix_rank(N)
print(f"shape of x_train: {X_train.shape}")
print(f"shape of y_train: {y_train.shape}")
print(f"shape of x_test: {X_test.shape}")
print(f"shape of y_test: {y_test.shape}")
print(f"X[0] Max, Min pre normalization: {np.max(X_train[:,0]):0.2f}, {np.min(X_train[:,0]):0.2f}")

tensor_x_train = torch.nn.functional.normalize(torch.Tensor(X_train)) # transform to torch tensor
tensor_y_train = torch.nn.functional.normalize(torch.Tensor(y_train))
train_dataset = TensorDataset(tensor_x_train,tensor_y_train) # create your datset
train_dataloader = DataLoader(train_dataset) # create your dataloader

tensor_x_test = torch.Tensor(X_test) # transform to torch tensor
tensor_y_test = torch.Tensor(y_test)
test_dataset = TensorDataset(tensor_x_test,tensor_y_test) # create your datset
test_dataloader = DataLoader(test_dataset) # create your dataloader

print(tensor_x_train[1,:])
print(f"X[0] Max, Min post normalization: {np.max(tensor_x_train.numpy()[:,0]):0.2f}, {np.min(tensor_x_train.numpy()[:,0]):0.2f}")

shape of x_train: (5000, 12)
shape of y_train: (5000, 42)
shape of x_test: (5000, 12)
shape of y_test: (5000, 42)
X[0] Max, Min pre normalization: 1.01, -1.11
tensor([-0.1418, -0.3170, -0.0433, -0.2664,  0.3101,  0.3541, -0.3976, -0.3660,
         0.0215,  0.2519, -0.2497,  0.4083])
X[0] Max, Min post normalization: 0.49, -0.53


Build Nueral-network

In [24]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.PN = PN
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(12, 64),
            nn.ReLU(),
            nn.Linear(64, 64),
            nn.ReLU(),
            nn.Linear(64, 64),
            nn.ReLU(),
            nn.Linear(64, m_vr),
        )

    def forward(self, x):
        x = self.flatten(x)
        v_r = self.linear_relu_stack(x)
        #print(torch.transpose(v_r, 0, 1))
        logits = (torch.matmul(self.PN, torch.transpose(v_r, 0, 1))+Pr0).T
        return logits

In [25]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=12, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=64, bias=True)
    (5): ReLU()
    (6): Linear(in_features=64, out_features=26, bias=True)
  )
)


In [26]:
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

In [27]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            #print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [31]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    start_time = time.time()
    train(train_dataloader, model, loss_fn, optimizer)
    print("--- %.2f seconds ---" % (time.time() - start_time))
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
--- 8.19 seconds ---
Test Error: 
 Accuracy: 0.0%, Avg loss: 0.425179 

Epoch 2
-------------------------------
--- 7.89 seconds ---
Test Error: 
 Accuracy: 0.0%, Avg loss: 0.425058 

Epoch 3
-------------------------------
--- 8.06 seconds ---
Test Error: 
 Accuracy: 0.0%, Avg loss: 0.424940 

Epoch 4
-------------------------------
--- 8.77 seconds ---
Test Error: 
 Accuracy: 0.0%, Avg loss: 0.424814 

Epoch 5
-------------------------------
--- 8.28 seconds ---
Test Error: 
 Accuracy: 0.0%, Avg loss: 0.424728 

Done!
